# 🤖 Hands-On: Machine Learning Lifecycle mit MLflow
**Szenario:** Wir sind Data Scientists bei einer Immobilienfirma. Unser Ziel ist es, ein KI-Modell zu trainieren, das basierend auf **Lage**, **Größe**, **Qualität** und **Baujahr** den Preis eines Hauses schätzt.

**Der Ablauf:**
1.  🧪 **Training (Experiments):** Wir trainieren das Modell und tracken Metriken.
2.  📚 **Registrierung (Models):** Wir speichern das beste Modell sauber ab.
3.  🔮 **Vorhersage (Inference):** Wir nutzen das Modell für neue Schätzungen.

In [0]:
# --- SCHRITT 1: DATEN VORBEREITEN ---
# Wir erstellen einen kleinen Datensatz (Ames Housing Dataset Auszug)

table_name = "workspace.default.house_prices"

print(f"🔧 Erstelle Trainingstabelle {table_name}...")

data = [
    (1, "CollgCr", 1710, 7, 2003, 208500.0),
    (2, "Veenker", 1262, 6, 1976, 181500.0),
    (3, "CollgCr", 1786, 7, 2001, 223500.0),
    (4, "Crawfor", 1717, 7, 1915, 140000.0),
    (5, "NoRidge", 2198, 8, 2000, 250000.0),
    (6, "Mitchel", 1362, 5, 1993, 143000.0)
]

columns = ["Id", "Neighborhood", "GrLivArea", "OverallQual", "YearBuilt", "SalePrice"]

# Overwrite sorgt dafür, dass wir das Notebook mehrfach starten können
df = spark.createDataFrame(data, columns)
df.write.format("delta").mode("overwrite").saveAsTable(table_name)

print("✅ Daten bereitgestellt.")
display(df)

🔧 Erstelle Trainingstabelle workspace.default.house_prices...
✅ Daten bereitgestellt.


Id,Neighborhood,GrLivArea,OverallQual,YearBuilt,SalePrice
1,CollgCr,1710,7,2003,208500.0
2,Veenker,1262,6,1976,181500.0
3,CollgCr,1786,7,2001,223500.0
4,Crawfor,1717,7,1915,140000.0
5,NoRidge,2198,8,2000,250000.0
6,Mitchel,1362,5,1993,143000.0


In [0]:
# --- SCHRITT 2: TRAINING (DAS LABOR) ---
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# 1. Daten laden
df_spark = spark.table(table_name)
dataset = df_spark.toPandas()

# TRICK: Da wir nur 6 Zeilen haben, vervielfachen wir die Daten künstlich (20x),
# damit der Algorithmus genug "Futter" für den Train/Test-Split hat.
dataset = pd.concat([dataset] * 20, ignore_index=True)

# 2. Encoding (Text -> Zahl)
# Computer können nicht mit "CollgCr" rechnen, nur mit Zahlen (0, 1, 2...)
le_neighborhood = LabelEncoder()
dataset['Neighborhood_encoded'] = le_neighborhood.fit_transform(dataset['Neighborhood'])

# Features (X) & Target (y)
X = dataset[['Neighborhood_encoded', 'GrLivArea', 'OverallQual', 'YearBuilt']]
y = dataset['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. MLflow Setup (Sicherer User-Abruf für Free Edition)
username = spark.sql("SELECT current_user()").first()[0]
experiment_path = f"/Users/{username}/House_Price_Predictor"

mlflow.set_experiment(experiment_path)
mlflow.sklearn.autolog() # <--- Die Magie: Loggt automatisch Params & Metriken

print(f"🚀 Starte Training in Experiment: {experiment_path}")

with mlflow.start_run(run_name="House_Price_Regressor_v1") as run:
    rf = RandomForestRegressor(n_estimators=50, random_state=42)
    rf.fit(X_train, y_train)
    
    # WICHTIG: Das Mapping speichern, damit wir später wissen, was 0, 1, 2 bedeutet!
    mapping = dict(zip(le_neighborhood.classes_, le_neighborhood.transform(le_neighborhood.classes_)))
    print("-" * 30)
    print("📌 MERKE DIR DIESES MAPPING FÜR SPÄTER:")
    print(mapping)
    print("-" * 30)

print(f"✅ Training beendet! Run ID: {run.info.run_id}")

INFO:py4j.clientserver:Received command c on object id p0


🚀 Starte Training in Experiment: /Users/philippe.christen@fhnw.ch/House_Price_Predictor


2025/11/26 10:43:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/11/26 10:43:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.12/site-packages/ml

------------------------------
📌 MERKE DIR DIESES MAPPING FÜR SPÄTER:
{'CollgCr': np.int64(0), 'Crawfor': np.int64(1), 'Mitchel': np.int64(2), 'NoRidge': np.int64(3), 'Veenker': np.int64(4)}
------------------------------
✅ Training beendet! Run ID: 4a55a2947da94c7d994681315ca9b4bc


In [0]:
# --- SCHRITT 3: REGISTRIERUNG (DER TRESOR) ---
# Wir automatisieren den Schritt "Register Model", den man sonst klickt.

import time

model_name = "workspace.default.house_price_estimator"
print(f"🔎 Suche neuesten Run im Experiment...")

# Neuesten Run holen
current_experiment = mlflow.get_experiment_by_name(experiment_path)
runs = mlflow.search_runs(experiment_ids=[current_experiment.experiment_id], 
                          order_by=["start_time DESC"], 
                          max_results=1)

if len(runs) > 0:
    latest_run_id = runs.iloc[0]["run_id"]
    print(f"📝 Registriere Modell aus Run {latest_run_id} als '{model_name}'...")
    
    model_uri = f"runs:/{latest_run_id}/model"
    mlflow.register_model(model_uri, model_name)
    
    print("⏳ Warte auf Modell-Bereitstellung...", end="")
    for _ in range(5):
        time.sleep(1)
        print(".", end="")
    print("\n✅ Modell ist registriert und bereit!")
else:
    print("❌ Fehler: Kein Training gefunden.")

🔎 Suche neuesten Run im Experiment...
📝 Registriere Modell aus Run 4a55a2947da94c7d994681315ca9b4bc als 'workspace.default.house_price_estimator'...


Registered model 'workspace.default.house_price_estimator' already exists. Creating a new version of this model...
Created version '2' of model 'workspace.default.house_price_estimator'.


⏳ Warte auf Modell-Bereitstellung........
✅ Modell ist registriert und bereit!


INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


In [0]:
# --- SCHRITT 4: VORHERSAGE (DIE ANWENDUNG) ---
# Jetzt nutzen wir das Modell wie eine App.

from mlflow import MlflowClient

client = MlflowClient()
print(f"🔎 Lade neueste Version von '{model_name}'...")

# Neueste Version finden (Unity Catalog kompatibel)
versions = client.search_model_versions(f"name='{model_name}'")
versions.sort(key=lambda x: int(x.version), reverse=True)
latest_version = versions[0].version

# Modell laden
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{latest_version}")

# --- SZENARIO: Ein neues Haus kommt auf den Markt ---
# Neighborhood Code: Siehe Mapping oben! (z.B. 0 = CollgCr, 2 = NoRidge)
neue_haus_daten = pd.DataFrame({
    "Neighborhood_encoded": [2],      # 2 = NoRidge (Teure Gegend!)
    "GrLivArea":            [2200],   # Großes Haus
    "OverallQual":          [8],      # Hohe Qualität
    "YearBuilt":            [2015]    # Modern
})

# Vorhersage
schätzung = loaded_model.predict(neue_haus_daten)[0]

print(f"\n🏠 HAUS-ANALYSE:")
print(f"   Lage-Code: {neue_haus_daten['Neighborhood_encoded'][0]}")
print(f"   Größe:     {neue_haus_daten['GrLivArea'][0]} sqft")
print(f"   Baujahr:   {neue_haus_daten['YearBuilt'][0]}")
print(f"💰 KI-SCHÄTZUNG: ${schätzung:,.2f}")

🔎 Lade neueste Version von 'workspace.default.house_price_estimator'...

🏠 HAUS-ANALYSE:
   Lage-Code: 2
   Größe:     2200 sqft
   Baujahr:   2015
💰 KI-SCHÄTZUNG: $241,540.00


# Cleanup

In [0]:
# --- CLEANUP SCRIPT ---
# Löscht alle Artefakte (Modell, Experiment, Tabelle).

# ⚠️ SICHERHEITSSCHALTER: Setze dies auf True, um wirklich zu löschen!
CLEANUP_NOW = False 

if CLEANUP_NOW:
    import mlflow
    from mlflow import MlflowClient

    # 1. Konfiguration
    model_name = "workspace.default.house_price_estimator"
    table_name = "workspace.default.house_prices"
    
    # Username sicher abrufen
    username = spark.sql("SELECT current_user()").first()[0]
    experiment_name = f"/Users/{username}/House_Price_Predictor"

    client = MlflowClient()

    print("🧹 Starte Aufräumen der ML-Umgebung...")

    # --- SCHRITT 1: Modell entfernen ---
    try:
        print(f"1️⃣ Prüfe Modell '{model_name}'...")
        versions = client.search_model_versions(f"name='{model_name}'")
        if versions:
            for v in versions:
                client.delete_model_version(name=model_name, version=v.version)
            client.delete_registered_model(name=model_name)
            print("   ✅ Modell komplett gelöscht.")
        else:
            print("   ℹ️ Modell war schon gelöscht.")
    except Exception as e:
        print(f"   ⚠️ Warnung (Modell): {e}")

    # --- SCHRITT 2: Experiment löschen ---
    try:
        print(f"2️⃣ Lösche Experiment '{experiment_name}'...")
        experiment = client.get_experiment_by_name(experiment_name)
        if experiment:
            client.delete_experiment(experiment.experiment_id)
            print("   ✅ Experiment gelöscht.")
        else:
            print("   ℹ️ Experiment war schon gelöscht.")
    except Exception as e:
        print(f"   ⚠️ Warnung (Experiment): {e}")

    # --- SCHRITT 3: Tabelle löschen ---
    # Ich habe die Kommentare entfernt, damit wirklich aufgeräumt wird
    print(f"3️⃣ Lösche Tabelle '{table_name}'...")
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    print("   ✅ Tabelle gelöscht.")

    print("\n🏁 System ist wieder sauber!")

else:
    print("ℹ️ Cleanup übersprungen.")
    print("   Setze die Variable CLEANUP_NOW = True im Code oben, um aufzuräumen.")

ℹ️ Cleanup übersprungen.
   Setze die Variable CLEANUP_NOW = True im Code oben, um aufzuräumen.
